## Descargar datos

In [3]:
!wget https://gist.githubusercontent.com/trantuyen082001/1fc2f5c0ad1507f40e721e6d18b34138/raw/56c3ca73768ceb74cdf6aa20ee7314c47d6ae08e/heart.csv -O data/original/heart.csv

--2025-10-03 23:58:10--  https://gist.githubusercontent.com/trantuyen082001/1fc2f5c0ad1507f40e721e6d18b34138/raw/56c3ca73768ceb74cdf6aa20ee7314c47d6ae08e/heart.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11019 (11K) [text/plain]
Saving to: ‘data/original/heart.csv’

100%[======================================>] 11,019      --.-K/s   in 0s      

2025-10-03 23:58:10 (90.3 MB/s) - ‘data/original/heart.csv’ saved [11019/11019]



## Leer datos y reorganizar

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv("data/original/heart.csv")

In [7]:
last_colum = df.pop("output")

In [9]:
df.insert(0, "target", last_colum)

In [12]:
df

,target,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,1,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,1,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,1,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,1,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,0,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,0,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,0,57,1,0,130,131,0,1,115,1,1.2,1,1,3


## Dividir en train y test

In [15]:
train, test = train_test_split(df, test_size=0.3)

In [16]:
train

,target,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
231,0,57,1,0,165,289,1,0,124,0,1.0,1,3,3
302,0,57,0,1,130,236,0,0,174,0,0.0,1,1,2
141,1,43,1,0,115,303,0,1,181,0,1.2,1,0,2
238,0,77,1,0,125,304,0,0,162,1,0.0,2,3,2
94,1,45,0,1,112,160,0,1,138,0,0.0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,1,39,0,2,138,220,0,1,152,0,0.0,1,0,2
174,0,60,1,0,130,206,0,0,132,1,2.4,1,2,3
13,1,64,1,3,110,211,0,0,144,1,1.8,1,0,2
65,1,35,0,0,138,183,0,1,182,0,1.4,2,0,2


In [19]:
train.to_csv("data/train.csv", header=None, index=None)

In [20]:
test.to_csv("data/test.csv", header=None, index=None)

In [22]:
!aws s3 cp data/train.csv s3://sagemaker-data-heart-attack-gfbg/heart/train.csv

upload: data/train.csv to s3://sagemaker-data-heart-attack-gfbg/heart/train.csv


In [23]:
!aws s3 cp data/test.csv s3://sagemaker-data-heart-attack-gfbg/heart/test.csv

upload: data/test.csv to s3://sagemaker-data-heart-attack-gfbg/heart/test.csv


## Preparación datos Gxboost

In [31]:
from sagemaker.inputs import TrainingInput
from sagemaker import image_uris
from sagemaker.estimator import Estimator

In [33]:
hyperparameters = {
        "max_depth":"5",
        "objective":"binary:logistic",
        "num_round":"50"}

In [25]:
train_input = TrainingInput("s3://sagemaker-data-heart-attack-gfbg/heart/train.csv", content_type="text/csv")

In [26]:
test_input = TrainingInput("s3://sagemaker-data-heart-attack-gfbg/heart/test.csv", content_type="text/csv")

## Crear estimador

In [29]:
container = image_uris.retrieve("xgboost", "us-east-1", "latest")

In [34]:
estimador = Estimator(image_uri=container, role="LabRole", 
                      instance_count=1, 
                      instance_type="ml.m5.xlarge", 
                      hyperparameters=hyperparameters)

In [36]:
estimador.fit({'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: xgboost-2025-10-04-00-58-53-999


2025-10-04 00:58:58 Starting - Starting the training job...
2025-10-04 00:59:13 Starting - Preparing the instances for training...
2025-10-04 00:59:58 Downloading - Downloading the training image......
2025-10-04 01:00:47 Training - Training image download completed. Training in progress..Arguments: train
[2025-10-04:01:00:55:INFO] Running standalone xgboost training.
[2025-10-04:01:00:55:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8075.66mb
[2025-10-04:01:00:55:INFO] Determined delimiter of CSV input is ','
[01:00:55] S3DistributionType set as FullyReplicated
[01:00:55] 212x13 matrix with 2756 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-10-04:01:00:55:INFO] Determined delimiter of CSV input is ','
[01:00:55] S3DistributionType set as FullyReplicated
[01:00:55] 91x13 matrix with 1183 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[01:00:55] src/tree/upd

## Realizar deploy

In [37]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

In [38]:
deployed = estimador.deploy(initial_instance_count=1, 
                            instance_type="ml.m5.xlarge",
                            serializer=CSVSerializer(),
                            deserializer=CSVDeserializer())

INFO:sagemaker:Creating model with name: xgboost-2025-10-04-01-28-22-094
INFO:sagemaker:Creating endpoint-config with name xgboost-2025-10-04-01-28-22-094
INFO:sagemaker:Creating endpoint with name xgboost-2025-10-04-01-28-22-094


-----!